### Sub_hot_1 Sub_hot_4 Sub_xgb_1 gbm_2 gbm_3

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
train_data = pd.read_json('train_data.json',orient="index")
test_data = pd.read_json('test_data.json',orient='index')

In [3]:
#set index
train_data.reset_index(inplace = True)
train_data.rename(columns={'index':'ID'}, inplace=True)

In [4]:
test_data.reset_index(inplace = True)
test_data.rename(columns={'index':'ID'}, inplace=True)

In [1]:
train_data.head(2)

NameError: name 'train_data' is not defined

In [6]:
train_data.titles[5]

u'India vs Australia 2nd Test English:2836,SRH vs RCB:406,KKR vs KXIP:177,MI vs SRH:502,Dominant India Win Series in Style:427,Lyon Pulls Oz Back on Day 2:29,Kohli Leads the Way with 122:82,1st ODI: India Begin Kohli Era with Win:512,All You Need to Know About One-off Test:40,India vs Australia 3rd Test English:459,India vs England | 2nd ODI | English:1198,India vs Bangladesh Day 3 English:374,Ruthless India Stay Perfect at Home:700,India Begin Kohli Era with Win:1022,RCB vs DD:1768,India 1st Innings: Fall of Wickets:420,Day 3: India Finally Break Oz Spirit:317,Aus Turn the Tide to Humble India:14,Bodyguard:998,RPS vs DD:244,MS Dhoni: The Untold Story:8786,Kaabil:9,KXIP vs RPS:3370,Lyon Rips Out India with 5-for:16,India vs Bangladesh Day 1 English:2535,India vs Australia 1st Test English:1453,RCB vs MI:4432,India vs England 2nd T20I English:7720,Ind Win Thriller Keep Series Alive:947,India vs Australia 4th Test English:2244,India Win Kohli Creates History:147,Resilient India on Top af

In [7]:
test_data.head(2)

,ID,cities,dow,genres,titles,tod
0,test-1,"delhi:182,mumbai:702","2:182,4:701","Romance:182,Cricket:702","Axar's 21-ball 40 amidst KXIP ruins:72,Virat K...","13:182,15:701"
1,test-10,"pune:16,delhi:1100,navi mumbai:22736","1:4830,3:3078,2:8359,5:2946,4:4185,7:451","Romance:50,Action:3501,Mythology:20300","Besh Korechi Prem Korechi:0,Power:122,Siya Ke ...","20:6909,21:1520,22:662,23:3160,19:5711,1:121,0..."


In [8]:
#check data
print ('Train data has {} rows and {} columns'.format(train_data.shape[0],train_data.shape[1]))
print ('test_data data has {} rows and {} columns'.format(test_data.shape[0],test_data.shape[1]))

Train data has 200000 rows and 7 columns
test_data data has 100000 rows and 6 columns


In [9]:
#Encode Target Variable
train_data = train_data.replace({'segment':{'pos':1,'neg':0}})

In [10]:
#check target variable count
train_data['segment'].value_counts()/train_data.shape[0]

0    0.923725
1    0.076275
Name: segment, dtype: float64

In [11]:
train_data['g1'] = [re.sub(pattern='\:\d+',repl='',string=x) for x in train_data['genres']]

In [12]:
train_data.g1[0:5]

0    Cricket,Kabaddi,Reality
1           Cricket,Wildlife
2            LiveTV,Football
3           TalkShow,Cricket
4      Drama,Cricket,Reality
Name: g1, dtype: object

In [13]:
train_data['g1'] = train_data['g1'].apply(lambda x: x.split(','))

In [14]:
train_data['g2'] = [re.sub(pattern='\:\d+', repl='', string = x) for x in train_data['dow']]
train_data['g2'] = train_data['g2'].apply(lambda x: x.split(','))

In [15]:
train_data.g2[0:5]

0    [1, 3, 2, 5, 4, 7, 6]
1    [1, 3, 2, 5, 4, 7, 6]
2                      [3]
3          [1, 3, 5, 4, 7]
4          [1, 2, 4, 7, 6]
Name: g2, dtype: object

In [16]:
t1 = pd.Series(train_data['g1']).apply(frozenset).to_frame(name='t_genre')

In [17]:
t1[0:5]

,t_genre
0,"(Cricket, Kabaddi, Reality)"
1,"(Cricket, Wildlife)"
2,"(LiveTV, Football)"
3,"(TalkShow, Cricket)"
4,"(Drama, Cricket, Reality)"


In [18]:
t2 = pd.Series(train_data['g2']).apply(frozenset).to_frame(name='t_dow')

In [19]:
# using frozenset trick - might take few minutes to process
for t_genre in frozenset.union(*t1.t_genre):
    t1[t_genre] = t1.apply(lambda _: int(t_genre in _.t_genre), axis=1)

In [20]:
t1[0:3]

,t_genre,Travel,Boxing,Crime,Romance,LiveTV,Hockey,FormulaE,Comedy,Teen,...,Drama,Action,Athletics,Reality,Documentary,Swimming,Formula1,Family,Badminton,Sport
0,"(Cricket, Kabaddi, Reality)",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,"(Cricket, Wildlife)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(LiveTV, Football)",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
for t_dow in frozenset.union(*t2.t_dow):
    t2[t_dow] = t2.apply(lambda _: int(t_dow in _.t_dow), axis = 1)

In [22]:
train_data = pd.concat([train_data.reset_index(drop=True), t1], axis=1)
train_data = pd.concat([train_data.reset_index(drop=True), t2], axis=1)

In [23]:
test_data['g1'] = [re.sub(pattern='\:\d+',repl='',string=x) for x in test_data['genres']]
test_data['g1'] = test_data['g1'].apply(lambda x: x.split(','))

test_data['g2'] = [re.sub(pattern='\:\d+', repl='', string = x) for x in test_data['dow']]
test_data['g2'] = test_data['g2'].apply(lambda x: x.split(','))

t1_te = pd.Series(test_data['g1']).apply(frozenset).to_frame(name='t_genre')
t2_te = pd.Series(test_data['g2']).apply(frozenset).to_frame(name='t_dow')

In [24]:
for t_genre in frozenset.union(*t1_te.t_genre):
    t1_te[t_genre] = t1_te.apply(lambda _: int(t_genre in _.t_genre), axis=1)

In [25]:
for t_dow in frozenset.union(*t2_te.t_dow):
    t2_te[t_dow] = t2_te.apply(lambda _: int(t_dow in _.t_dow), axis = 1)

In [26]:
test_data = pd.concat([test_data.reset_index(drop=True), t1_te], axis=1)
test_data = pd.concat([test_data.reset_index(drop=True), t2_te], axis=1)

In [27]:
#the rows aren't list exactly. They are object, so we convert them to list and extract the watch time
w1 = train_data['titles']
w1 = w1.str.split(',')

#create a nested list of numbers
main = []
for i in np.arange(train_data.shape[0]):
    d1 = w1[i]
    nest = []
    nest = [re.sub(pattern = '.*\:', repl=' ', string= d1[k]) for k in list(np.arange(len(d1)))]
    main.append(nest)

In [28]:
main[2]

[u' 13', u' 4129']

In [29]:
blanks = []
for i in np.arange(len(main)):
    if '' in main[i]:
        print "{} blanks found".format(len(blanks))
        blanks.append(i)
        
#replacing blanks with 0
for i in blanks:
    main[i] = [x.replace('','0') for x in main[i]]
    
#converting string to integers
main = [[int(y) for y in x] for x in main]

#adding the watch time
tosum = []
for i in np.arange(len(main)):
    s = sum(main[i])
    tosum.append(s)

0 blanks found
1 blanks found
2 blanks found
3 blanks found
4 blanks found
5 blanks found
6 blanks found
7 blanks found
8 blanks found
9 blanks found
10 blanks found
11 blanks found
12 blanks found
13 blanks found
14 blanks found
15 blanks found
16 blanks found
17 blanks found
18 blanks found
19 blanks found
20 blanks found
21 blanks found
22 blanks found
23 blanks found
24 blanks found
25 blanks found
26 blanks found
27 blanks found
28 blanks found
29 blanks found
30 blanks found
31 blanks found
32 blanks found
33 blanks found
34 blanks found
35 blanks found
36 blanks found
37 blanks found
38 blanks found
39 blanks found
40 blanks found
41 blanks found
42 blanks found
43 blanks found
44 blanks found
45 blanks found


In [30]:
tosum[0]

87363

In [31]:
train_data['title_sum'] = tosum

In [32]:
#making changes in test data
w1_te = test_data['titles']
w1_te = w1_te.str.split(',')

In [33]:
main_te = []
for i in np.arange(test_data.shape[0]):
    d1 = w1_te[i]
    nest = []
    nest = [re.sub(pattern = '.*\:', repl=' ', string= d1[k]) for k in list(np.arange(len(d1)))]
    main_te.append(nest)

In [34]:
blanks_te = []
for i in np.arange(len(main_te)):
    if '' in main_te[i]:
        print "{} blanks found".format(len(blanks_te))
        blanks_te.append(i)
        
#replacing blanks with 0
for i in blanks_te:
    main_te[i] = [x.replace('','0') for x in main_te[i]]
    
#converting string to integers
main_te = [[int(y) for y in x] for x in main_te]

#adding the watch time
tosum_te = []
for i in np.arange(len(main_te)):
    s = sum(main_te[i])
    tosum_te.append(s)

0 blanks found
1 blanks found
2 blanks found
3 blanks found
4 blanks found
5 blanks found
6 blanks found
7 blanks found
8 blanks found
9 blanks found
10 blanks found
11 blanks found


In [35]:
test_data['title_sum'] = tosum_te

In [36]:
#count variables
def wcount(p):
    return p.count(',')+1

In [37]:
train_data['title_count'] = train_data['titles'].map(wcount)
train_data['genres_count'] = train_data['genres'].map(wcount)
train_data['cities_count'] = train_data['cities'].map(wcount)
train_data['tod_count'] = train_data['tod'].map(wcount)
train_data['dow_count'] = train_data['dow'].map(wcount)


test_data['title_count'] = test_data['titles'].map(wcount)
test_data['genres_count'] = test_data['genres'].map(wcount)
test_data['cities_count'] = test_data['cities'].map(wcount)
test_data['tod_count'] = test_data['tod'].map(wcount)
test_data['dow_count'] = test_data['dow'].map(wcount)

In [38]:
train_data.tod[0]

u'10:26,13:331,12:323,20:21864,21:16233,17:7953,16:1725,19:11505,18:6017,22:8894,2:2363,14:2196,15:2901,23:5046'

In [39]:
test_id = test_data['ID']
train_data.drop(['ID','cities','dow','genres','titles','tod','g1','g2','t_genre','t_dow'], inplace=True, axis=1)
test_data.drop(['ID','cities','dow','genres','titles','tod','g1','g2','t_genre','t_dow'], inplace=True, axis=1)

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [41]:
target = train_data['segment']
train_data.drop('segment',axis=1, inplace=True)

In [46]:
#train final model
rf_model = RandomForestClassifier(n_estimators=500,max_depth=12, max_features=10)
rf_model.fit(train_data, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features=10, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [47]:
#make prediction
rf_pred = rf_model.predict_proba(test_data)

In [48]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_hot.csv", index=False)

In [49]:
train_data.head(2)

,Travel,Boxing,Crime,Romance,LiveTV,Hockey,FormulaE,Comedy,Teen,Cricket,...,5,4,7,6,title_sum,title_count,genres_count,cities_count,tod_count,dow_count
0,0,0,0,0,0,0,0,0,0,1,...,1,1,1,1,87363,60,3,2,14,7
1,0,0,0,0,0,0,0,0,0,1,...,1,1,1,1,16356,70,2,3,16,7


In [50]:
train_data.columns

Index([      u'Travel',       u'Boxing',        u'Crime',      u'Romance',
             u'LiveTV',       u'Hockey',     u'FormulaE',       u'Comedy',
               u'Teen',      u'Cricket',    u'Mythology',           u'NA',
             u'Horror',     u'Football',       u'Awards',      u'Science',
             u'Tennis',     u'Thriller',      u'Kabaddi',     u'Wildlife',
               u'Kids',    u'IndiaVsSa',     u'TalkShow', u'Table Tennis',
         u'Volleyball',        u'Drama',       u'Action',    u'Athletics',
            u'Reality',  u'Documentary',     u'Swimming',     u'Formula1',
             u'Family',    u'Badminton',        u'Sport',            u'1',
                  u'3',            u'2',            u'5',            u'4',
                  u'7',            u'6',    u'title_sum',  u'title_count',
       u'genres_count', u'cities_count',    u'tod_count',    u'dow_count'],
      dtype='object')

In [51]:
test_data.columns

Index([      u'Travel',      u'Kabaddi',      u'Reality',      u'Romance',
             u'LiveTV',       u'Hockey',     u'FormulaE',       u'Comedy',
               u'Teen',      u'Cricket',           u'NA',       u'Horror',
           u'Football',       u'Awards',      u'Science',       u'Tennis',
           u'Thriller',       u'Boxing',     u'Wildlife',         u'Kids',
          u'IndiaVsSa',     u'TalkShow', u'Table Tennis',   u'Volleyball',
              u'Drama',       u'Action',    u'Athletics',    u'Mythology',
        u'Documentary',     u'Swimming',     u'Formula1',       u'Family',
          u'Badminton',        u'Crime',        u'Sport',            u'1',
                  u'3',            u'2',            u'5',            u'4',
                  u'7',            u'6',    u'title_sum',  u'title_count',
       u'genres_count', u'cities_count',    u'tod_count',    u'dow_count'],
      dtype='object')

In [71]:
import seaborn as sns

corr = train_data.corr()

# plot the heatmap
sns.heatmap(corr, 
        xticklabels=train_data.columns,
        yticklabels=train_data.columns)

ValueError: Arrays were different lengths: 49 vs 0

In [72]:
corr

,Travel,Boxing,Crime,Romance,LiveTV,Hockey,FormulaE,Comedy,Teen,Cricket,...,4,7,6,title_sum,title_count,genres_count,cities_count,tod_count,dow_count,Sports
Travel,1.000000,0.007573,0.054673,0.025045,0.021923,0.023399,0.009057,0.062497,0.025410,0.044333,...,0.039497,0.047539,0.033710,0.034997,0.112009,0.208804,0.039566,0.068528,0.062677,0.064003
Boxing,0.007573,1.000000,0.002803,0.000656,-0.000862,0.009051,-0.000146,0.007163,-0.001164,0.006339,...,0.004957,0.004540,0.003534,0.000667,0.009830,0.017302,0.005447,0.005619,0.006576,0.032543
Crime,0.054673,0.002803,1.000000,0.043393,0.026896,0.027738,-0.000939,0.070713,0.037093,0.060111,...,0.080793,0.085362,0.082031,0.110798,0.153056,0.288791,0.065593,0.144315,0.125688,0.067306
Romance,0.025045,0.000656,0.043393,1.000000,-0.014271,0.015228,0.001802,0.075592,0.074046,-0.107897,...,0.166997,0.161590,0.181664,0.260023,0.153745,0.407997,0.134972,0.305359,0.281851,-0.076503
LiveTV,0.021923,-0.000862,0.026896,-0.014271,1.000000,0.025226,0.008171,-0.001244,-0.001504,0.185237,...,0.115022,0.111266,0.111330,0.027082,0.180399,0.238509,-0.015676,0.140583,0.154411,0.172253
Hockey,0.023399,0.009051,0.027738,0.015228,0.025226,1.000000,0.007795,0.027476,0.004633,0.070411,...,0.048951,0.057249,0.044358,0.052779,0.182099,0.145005,0.053557,0.083772,0.074022,0.261669
FormulaE,0.009057,-0.000146,-0.000939,0.001802,0.008171,0.007795,1.000000,0.003932,-0.001312,0.007988,...,0.006457,0.005391,0.005276,0.009866,0.032267,0.025920,0.006928,0.010976,0.006278,0.044209
Comedy,0.062497,0.007163,0.070713,0.075592,-0.001244,0.027476,0.003932,1.000000,0.032361,0.035706,...,0.121930,0.127833,0.118561,0.172120,0.206029,0.411253,0.139881,0.235941,0.201953,0.074812
Teen,0.025410,-0.001164,0.037093,0.074046,-0.001504,0.004633,-0.001312,0.032361,1.000000,-0.017753,...,0.037250,0.038309,0.037283,0.122096,0.050393,0.132123,0.030598,0.088077,0.061481,-0.010986
Cricket,0.044333,0.006339,0.060111,-0.107897,0.185237,0.070411,0.007988,0.035706,-0.017753,1.000000,...,0.282839,0.248950,0.233317,0.041785,0.375934,0.348121,0.193332,0.275311,0.324484,0.857666


In [42]:
train_data["Sports"] = train_data.Boxing + train_data.Hockey + train_data.FormulaE + train_data.Cricket + train_data.Football + train_data.Tennis + train_data.Kabaddi + train_data.IndiaVsSa + train_data["Table Tennis"] + train_data.Volleyball + train_data.Athletics + train_data.Swimming + train_data.Formula1 + train_data.Badminton + train_data.Sport

In [43]:
test_data["Sports"] = test_data.Boxing + test_data.Hockey + test_data.FormulaE + test_data.Cricket + test_data.Football + test_data.Tennis + test_data.Kabaddi + test_data.IndiaVsSa + test_data["Table Tennis"] + test_data.Volleyball + test_data.Athletics + test_data.Swimming + test_data.Formula1 + test_data.Badminton + test_data.Sport

In [44]:
tt = train_data[["Travel","Reality","Romance","LiveTV","Comedy","Teen","NA","Horror","Awards","Science","Thriller",
               "Wildlife","Kids","TalkShow","Drama","Action","Mythology","Documentary","Family","Crime","Sports",
                "1","2","3","4","5","6","7","title_sum","title_count","genres_count","cities_count","tod_count",
                "dow_count"]]

In [45]:
tt.head(2)

,Travel,Reality,Romance,LiveTV,Comedy,Teen,NA,Horror,Awards,Science,...,4,5,6,7,title_sum,title_count,genres_count,cities_count,tod_count,dow_count
0,0,1,0,0,0,0,0,0,0,0,...,1,1,1,1,87363,60,3,2,14,7
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,16356,70,2,3,16,7


In [46]:
tttest = test_data[["Travel","Reality","Romance","LiveTV","Comedy","Teen","NA","Horror","Awards","Science","Thriller",
               "Wildlife","Kids","TalkShow","Drama","Action","Mythology","Documentary","Family","Crime","Sports",
                "1","2","3","4","5","6","7","title_sum","title_count","genres_count","cities_count","tod_count",
                "dow_count"]]

In [78]:
s = tt.title_sum.sum()/len(tt)
s

21222

In [79]:
tt.title_sum = (tt.title_sum-s)/s

/home/manishsihag/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [80]:
s = tttest.title_sum.sum()/len(tttest)
s

23194

In [81]:
tttest.title_sum = (tttest.title_sum-s)/s

In [82]:
tt.describe()

,Travel,Reality,Romance,LiveTV,Comedy,Teen,NA,Horror,Awards,Science,...,4,5,6,7,title_sum,title_count,genres_count,cities_count,tod_count,dow_count
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000
mean,0.020740,0.292110,0.357035,0.141765,0.234040,0.010320,0.000060,0.032925,0.058055,0.008905,...,0.637100,0.611025,0.61869,0.613865,0.000039,11.802520,3.329420,1.912525,7.853135,4.21007
std,0.142513,0.454734,0.479126,0.348810,0.423398,0.101062,0.007746,0.178441,0.233848,0.093945,...,0.480838,0.487519,0.48571,0.486863,2.173682,16.245102,2.089513,1.229760,5.527403,2.23663
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,1.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,-0.927387,3.000000,2.000000,1.000000,3.000000,2.00000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.00000,1.000000,-0.682782,6.000000,3.000000,1.000000,7.000000,4.00000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.00000,1.000000,0.002698,14.000000,4.000000,2.000000,12.000000,7.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,93.427057,464.000000,22.000000,22.000000,24.000000,7.00000


In [83]:
tt.columns

Index([      u'Travel',      u'Reality',      u'Romance',       u'LiveTV',
             u'Comedy',         u'Teen',           u'NA',       u'Horror',
             u'Awards',      u'Science',     u'Thriller',     u'Wildlife',
               u'Kids',     u'TalkShow',        u'Drama',       u'Action',
          u'Mythology',  u'Documentary',       u'Family',        u'Crime',
             u'Sports',            u'1',            u'2',            u'3',
                  u'4',            u'5',            u'6',            u'7',
          u'title_sum',  u'title_count', u'genres_count', u'cities_count',
          u'tod_count',    u'dow_count'],
      dtype='object')

In [84]:
#train final model
rf_model = RandomForestClassifier(n_estimators=500,max_depth=12, max_features=10)
rf_model.fit(tt, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features=10, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [85]:
#make prediction
rf_pred = rf_model.predict_proba(tttest)

In [86]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_hot_5.csv", index=False)

In [87]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=5, n_estimators=300, learning_rate=0.04).fit(tt, target)
pred = gbm.predict_proba(tttest)

In [88]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_xgboost_1.csv", index=False)

In [89]:
prd = 0.50*rf_pred+0.50*pred

In [90]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_50rf50xgb.csv", index=False)

In [91]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
    n_estimators= 2000,
    max_depth= 4,
    min_child_weight= 2,
    #gamma=1,
    gamma=0.9,                        
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(tt, target)
predictions = gbm.predict_proba(tttest)

In [103]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=predictions[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("gbm.csv", index=False)

In [104]:
gbm = xgb.XGBClassifier(
    learning_rate = 0.05,
    n_estimators= 300,
    max_depth= 4,
    min_child_weight= 2,
    gamma=1,
    #gamma=0.9,                        
    subsample=0.8,
    colsample_bytree=0.8,
    #objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(tt, target)
predictions_2 = gbm.predict_proba(tttest)

In [105]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=predictions_2[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("gbm_2.csv", index=False)

In [99]:
import lightgbm as lgb

In [114]:
ttdata=lgb.Dataset(tt,label=target)

#setting parameters for lightgbm
param = {'num_leaves':200, 'objective':'binary','max_depth':10,'learning_rate':.05,'max_bin':300}
param['metric'] = ['binary_logloss']

#Here we have set max_depth in xgb and LightGBM to 7 to have a fair comparison between the two.

#training our model using light gbm
num_round=100
lgbm=lgb.train(param,ttdata,num_round)

#predicting on test set
ypred=lgbm.predict(tttest)

In [115]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=ypred, columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("lightgbm_2.csv", index=False)

In [116]:
gbm = xgb.XGBClassifier(
    learning_rate = 0.05,
    n_estimators= 250,
    max_depth= 7,
    min_child_weight= 2,
    #gamma=1,
    gamma=0.9,                        
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(tt, target)
predictions_3 = gbm.predict_proba(tttest)

In [117]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=predictions_3[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("gbm_3.csv", index=False)

In [118]:
gbm = xgb.XGBClassifier(
    learning_rate = 0.08,
    n_estimators= 500,
    max_depth= 3,
    min_child_weight= 2,
    gamma=1,
    #gamma=0.9,                        
    subsample=0.8,
    colsample_bytree=0.8,
    #objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(tt, target)
predictions_4 = gbm.predict_proba(tttest)

In [119]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=predictions_4[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("gbm_4.csv", index=False)

In [120]:
gbm = xgb.XGBClassifier(
    learning_rate = 0.1,
    n_estimators= 1000,
    max_depth= 4,
    min_child_weight= 2,
    gamma=1,
    #gamma=0.9,                        
    subsample=0.6,
    colsample_bytree=0.4,
    #objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(tt, target)
predictions_5 = gbm.predict_proba(tttest)

In [121]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=predictions_5[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("gbm_5.csv", index=False)

In [131]:
PRED = (predictions[:,1]+predictions_2[:,1]+predictions_3[:,1]+predictions_4[:,1]+predictions_5[:,1]+ypred)+1

In [133]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=PRED, columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("PRED.csv", index=False)

In [52]:
train_data["Sports"] = train_data.Boxing + train_data.Hockey + train_data.Football + train_data.Tennis + train_data.Kabaddi + train_data["Table Tennis"] + train_data.Volleyball + train_data.Athletics + train_data.Swimming + train_data.Badminton

In [53]:
test_data["Sports"] = test_data.Boxing + test_data.Hockey + test_data.Football + test_data.Tennis + test_data.Kabaddi + test_data["Table Tennis"] + test_data.Volleyball + test_data.Athletics + test_data.Swimming + test_data.Badminton

In [54]:
train_data.columns

Index([      u'Travel',       u'Boxing',        u'Crime',      u'Romance',
             u'LiveTV',       u'Hockey',     u'FormulaE',       u'Comedy',
               u'Teen',      u'Cricket',    u'Mythology',           u'NA',
             u'Horror',     u'Football',       u'Awards',      u'Science',
             u'Tennis',     u'Thriller',      u'Kabaddi',     u'Wildlife',
               u'Kids',    u'IndiaVsSa',     u'TalkShow', u'Table Tennis',
         u'Volleyball',        u'Drama',       u'Action',    u'Athletics',
            u'Reality',  u'Documentary',     u'Swimming',     u'Formula1',
             u'Family',    u'Badminton',        u'Sport',            u'1',
                  u'3',            u'2',            u'5',            u'4',
                  u'7',            u'6',    u'title_sum',  u'title_count',
       u'genres_count', u'cities_count',    u'tod_count',    u'dow_count',
             u'Sports'],
      dtype='object')

In [55]:
train_data["History"] = train_data.Mythology+train_data.Awards+train_data.Science+train_data.Documentary
train_data["Child"] = train_data.Teen+train_data.Kids
train_data["Scary"] = train_data.Crime+train_data.Horror+train_data.Thriller+train_data.Action
train_data["Formula"] = train_data.Formula1 + train_data.FormulaE
train_data["Cric"] = train_data.Cricket+train_data.IndiaVsSa
train_data["Girls"] = train_data.Romance+train_data.Drama+train_data.Family

train_data["TravellingPerson"] = [0 for w in train_data.cities_count]
for i,w in enumerate(train_data.cities_count):
    if(w>2):
        train_data["TravellingPerson"][i] += 1

In [57]:
test_data["History"] = test_data.Mythology+test_data.Awards+test_data.Science+test_data.Documentary
test_data["Child"] = test_data.Teen+test_data.Kids
test_data["Scary"] = test_data.Crime+test_data.Horror+test_data.Thriller+test_data.Action
test_data["Formula"] = test_data.Formula1 + test_data.FormulaE
test_data["Cric"] = test_data.Cricket+test_data.IndiaVsSa
test_data["Girls"] = test_data.Romance+test_data.Drama+test_data.Family

test_data["TravellingPerson"] = [0 for w in test_data.cities_count]
for i,w in enumerate(test_data.cities_count):
    if(w>2):
        test_data["TravellingPerson"][i] += 1

In [58]:
tt = train_data

In [59]:
tt.columns

Index([          u'Travel',           u'Boxing',            u'Crime',
                u'Romance',           u'LiveTV',           u'Hockey',
               u'FormulaE',           u'Comedy',             u'Teen',
                u'Cricket',        u'Mythology',               u'NA',
                 u'Horror',         u'Football',           u'Awards',
                u'Science',           u'Tennis',         u'Thriller',
                u'Kabaddi',         u'Wildlife',             u'Kids',
              u'IndiaVsSa',         u'TalkShow',     u'Table Tennis',
             u'Volleyball',            u'Drama',           u'Action',
              u'Athletics',          u'Reality',      u'Documentary',
               u'Swimming',         u'Formula1',           u'Family',
              u'Badminton',            u'Sport',                u'1',
                      u'3',                u'2',                u'5',
                      u'4',                u'7',                u'6',
              u'titl

In [60]:
tt.drop(['Crime','Romance','FormulaE','Teen','Cricket','Mythology','Horror',
        'Awards','Science','Thriller','Kids','IndiaVsSa','Drama','Action',
        'Documentary','Formula1','Family'], inplace=True, axis=1)

In [61]:
tttest = test_data

In [62]:
tttest.columns

Index([          u'Travel',          u'Kabaddi',          u'Reality',
                u'Romance',           u'LiveTV',           u'Hockey',
               u'FormulaE',           u'Comedy',             u'Teen',
                u'Cricket',               u'NA',           u'Horror',
               u'Football',           u'Awards',          u'Science',
                 u'Tennis',         u'Thriller',           u'Boxing',
               u'Wildlife',             u'Kids',        u'IndiaVsSa',
               u'TalkShow',     u'Table Tennis',       u'Volleyball',
                  u'Drama',           u'Action',        u'Athletics',
              u'Mythology',      u'Documentary',         u'Swimming',
               u'Formula1',           u'Family',        u'Badminton',
                  u'Crime',            u'Sport',                u'1',
                      u'3',                u'2',                u'5',
                      u'4',                u'7',                u'6',
              u'titl

In [63]:
tttest.drop(['Crime','Romance','FormulaE','Teen','Cricket','Mythology','Horror',
        'Awards','Science','Thriller','Kids','IndiaVsSa','Drama','Action',
        'Documentary','Formula1','Family'], inplace=True, axis=1)

In [64]:
#train final model
rf_model = RandomForestClassifier(n_estimators=500,max_depth=12, max_features=10)
rf_model.fit(tt, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features=10, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [65]:
#make prediction
rf_pred = rf_model.predict_proba(tttest)

In [66]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_hot_3.csv", index=False)

In [60]:
trn = tt
trn["segment"] = target
tst = tttest
tst["ID"] = test_id

In [61]:
trn.to_csv("train_data.csv",index = False)
tst.to_csv("test_data.csv",index = False)

In [46]:
tt.columns

Index([      u'Travel',      u'Reality',      u'Romance',       u'LiveTV',
             u'Comedy',         u'Teen',           u'NA',       u'Horror',
             u'Awards',      u'Science',     u'Thriller',     u'Wildlife',
               u'Kids',     u'TalkShow',        u'Drama',       u'Action',
          u'Mythology',  u'Documentary',       u'Family',        u'Crime',
             u'Sports',            u'1',            u'2',            u'3',
                  u'4',            u'5',            u'6',            u'7',
          u'title_sum',  u'title_count', u'genres_count', u'cities_count',
          u'tod_count',    u'dow_count'],
      dtype='object')

In [54]:
tt.describe()

,Travel,Reality,Romance,LiveTV,Comedy,Teen,NA,Horror,Awards,Science,...,4,5,6,7,title_sum,title_count,genres_count,cities_count,tod_count,dow_count
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.00000,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000
mean,0.020740,0.292110,0.357035,0.141765,0.234040,0.010320,0.000060,0.032925,0.058055,0.008905,...,0.637100,0.611025,0.61869,0.613865,1.187439e-15,11.802520,3.329420,1.912525,7.853135,4.21007
std,0.142513,0.454734,0.479126,0.348810,0.423398,0.101062,0.007746,0.178441,0.233848,0.093945,...,0.480838,0.487519,0.48571,0.486863,2.173597e+00,16.245102,2.089513,1.229760,5.527403,2.23663
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,-1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,-9.273895e-01,3.000000,2.000000,1.000000,3.000000,2.00000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.00000,1.000000,-6.827945e-01,6.000000,3.000000,1.000000,7.000000,4.00000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.00000,1.000000,2.658375e-03,14.000000,4.000000,2.000000,12.000000,7.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,9.342336e+01,464.000000,22.000000,22.000000,24.000000,7.00000


In [52]:
mean = tt.title_sum.mean()

In [53]:
tt.title_sum = (tt.title_sum-mean)/mean

/home/manishsihag/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [55]:
mean = tttest.title_sum.mean()

In [56]:
tttest.title_sum = (tttest.title_sum-mean)/mean

In [57]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=350, learning_rate=0.03).fit(tt, target)
pred = gbm.predict_proba(tttest)

In [58]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_xgboost.csv", index=False)

In [69]:
prd = 0.80*rf_pred+0.20*pred

In [70]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_80rf20xgb.csv", index=False)